<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Guided Practice with Topic Modeling and LDA

_Authors: Dave Yerrington (SF)_

---

> **Note: this lab is intended to be a guided lab with the instructor.**

In practice it would be a very rare to need to build an unsupervised topic model like LDA from scratch. Lucky for us, sklearn comes with LDA(Latent Dirichlet Allocation) topic modeling functionality. Another popular LDA module which we will explore in this lab is from the `gensim` package. 

Let's explore a brief walkthrough of LDA and topic modeling using gensim. We will work with a small collection of documents represented as a list.

### 1. Load the packages and create the small "documents".

You may need to install the gensim package with `pip` or `conda`.

In [79]:
from gensim import corpora, models, matutils
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from collections import defaultdict
import pandas as pd
import numpy as np

doc_a = "Brocolli is good to eat. My brother likes to eat good brocolli, but not my mother."
doc_b = "My mother spends a lot of time driving my brother around to baseball practice."
doc_c = "Some health experts suggest that driving may cause increased tension and blood pressure."
doc_d = "I often feel pressure to perform well at school, but my mother never seems to drive my brother to do better."
doc_e = "Health professionals say that brocolli is good for your health."

# compile sample documents into a list
documents = [doc_a, doc_b, doc_c, doc_d, doc_e]
df        = pd.DataFrame(documents, columns=['text'])

In [20]:
df

,text
0,Brocolli is good to eat. My brother likes to e...
1,My mother spends a lot of time driving my brot...
2,Some health experts suggest that driving may c...
3,I often feel pressure to perform well at schoo...
4,Health professionals say that brocolli is good...


In [46]:
X = df.text

### 2. Load stop words either from NLTK or sklearn

In [50]:
import nltk
nltk.download('stopwords')
print(stopwords.words('english'))
len(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\13305\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


179

In [40]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
ENGLISH_STOP_WORDS

frozenset({'a',
           'about',
           'above',
           'across',
           'after',
           'afterwards',
           'again',
           'against',
           'all',
           'almost',
           'alone',
           'along',
           'already',
           'also',
           'although',
           'always',
           'am',
           'among',
           'amongst',
           'amoungst',
           'amount',
           'an',
           'and',
           'another',
           'any',
           'anyhow',
           'anyone',
           'anything',
           'anyway',
           'anywhere',
           'are',
           'around',
           'as',
           'at',
           'back',
           'be',
           'became',
           'because',
           'become',
           'becomes',
           'becoming',
           'been',
           'before',
           'beforehand',
           'behind',
           'being',
           'below',
           'beside',
           'besides'

In [51]:
# A:
len(ENGLISH_STOP_WORDS)

318

### 3. Use CountVectorizer to transform our text, taking out the stopwords.

In [47]:
# A:
vect1 = CountVectorizer(stop_words=stopwords.words('english'))
vect2 = CountVectorizer(stop_words=ENGLISH_STOP_WORDS)
df_dtm1 = vect1.fit_transform(X)
df_dtm2 = vect2.fit_transform(X)
# sklearn stopwords has more items than that of the NLTK, therefore eliminates more stopwords 
print(vect1.vocabulary_)
print(vect2.vocabulary_)

{'brocolli': 4, 'good': 12, 'eat': 9, 'brother': 5, 'likes': 15, 'mother': 18, 'spends': 28, 'lot': 16, 'time': 31, 'driving': 8, 'around': 0, 'baseball': 1, 'practice': 22, 'health': 13, 'experts': 10, 'suggest': 29, 'may': 17, 'cause': 6, 'increased': 14, 'tension': 30, 'blood': 3, 'pressure': 23, 'often': 20, 'feel': 11, 'perform': 21, 'well': 32, 'school': 26, 'never': 19, 'seems': 27, 'drive': 7, 'better': 2, 'professionals': 24, 'say': 25}
{'brocolli': 3, 'good': 11, 'eat': 8, 'brother': 4, 'likes': 14, 'mother': 16, 'spends': 23, 'lot': 15, 'time': 26, 'driving': 7, 'baseball': 0, 'practice': 18, 'health': 12, 'experts': 9, 'suggest': 24, 'cause': 5, 'increased': 13, 'tension': 25, 'blood': 2, 'pressure': 19, 'feel': 10, 'perform': 17, 'school': 22, 'drive': 6, 'better': 1, 'professionals': 20, 'say': 21}


### 4. Extract the tokens that remain after stopword removal.

The `.vocabulary_` attribute of the vectorizer contains a dictionary of terms. There is also the built-in function `.get_feature_names()` which will extract the column names.

In [53]:
# A:
print(vect1.get_feature_names())
len(vect1.get_feature_names())

['around', 'baseball', 'better', 'blood', 'brocolli', 'brother', 'cause', 'drive', 'driving', 'eat', 'experts', 'feel', 'good', 'health', 'increased', 'likes', 'lot', 'may', 'mother', 'never', 'often', 'perform', 'practice', 'pressure', 'professionals', 'say', 'school', 'seems', 'spends', 'suggest', 'tension', 'time', 'well']


33

In [54]:
print(vect2.get_feature_names())
len(vect2.get_feature_names())

['baseball', 'better', 'blood', 'brocolli', 'brother', 'cause', 'drive', 'driving', 'eat', 'experts', 'feel', 'good', 'health', 'increased', 'likes', 'lot', 'mother', 'perform', 'practice', 'pressure', 'professionals', 'say', 'school', 'spends', 'suggest', 'tension', 'time']


27

### 5. Get counts of tokens.

Convert the matrix from the vectorizer to a dense matrix, then sum by column to get the counts per term.

In [68]:
# A:
df_dtm1_arr = df_dtm1.toarray()
df1 = df_dtm1_arr.sum(axis=0)
df1=pd.DataFrame(df1.reshape(1,len(vect1.get_feature_names())), columns=vect1.get_feature_names())
df1

,around,baseball,better,blood,brocolli,brother,cause,drive,driving,eat,...,pressure,professionals,say,school,seems,spends,suggest,tension,time,well
0,1,1,1,1,3,3,1,1,2,2,...,2,1,1,1,1,1,1,1,1,1


In [69]:
df_dtm2_arr = df_dtm2.toarray()
df2 = df_dtm2_arr.sum(axis=0)
df2=pd.DataFrame(df2.reshape(1,len(vect2.get_feature_names())), columns=vect2.get_feature_names())
df2

,baseball,better,blood,brocolli,brother,cause,drive,driving,eat,experts,...,perform,practice,pressure,professionals,say,school,spends,suggest,tension,time
0,1,1,1,3,3,1,1,2,2,1,...,1,1,2,1,1,1,1,1,1,1


### 6. Setup the vocabulary dictionary

First we need to setup the vocabulary.  Gensim's LDA expects our vocabulary to be in a format where the dictionary keys are the column indices and the values are the words themselves.

Create this dictionary below.

In [85]:
# A:
vocab1 = {}
for i in range(len(vect1.get_feature_names())):
    vocab1[i] = vect1.get_feature_names()[i]
vocab2 = {}
for i in range(len(vect2.get_feature_names())):
    vocab2[i] = vect2.get_feature_names()[i]
vocab2

{0: 'baseball',
 1: 'better',
 2: 'blood',
 3: 'brocolli',
 4: 'brother',
 5: 'cause',
 6: 'drive',
 7: 'driving',
 8: 'eat',
 9: 'experts',
 10: 'feel',
 11: 'good',
 12: 'health',
 13: 'increased',
 14: 'likes',
 15: 'lot',
 16: 'mother',
 17: 'perform',
 18: 'practice',
 19: 'pressure',
 20: 'professionals',
 21: 'say',
 22: 'school',
 23: 'spends',
 24: 'suggest',
 25: 'tension',
 26: 'time'}

### 7. Create a token to id mapping with gensim's `corpora.Dictionary`

This dictionary class is a more standard way to work with with gensim models. There are a few standard steps we should go through:

**7.1. Count the frequency of words.**

We can do this easily with the python `defaultdict(int)`, which doesn't require us to already have the key in the dictionary to be able to add to it:

```python
frequency = defaultdict(int)

for text in documents:
    for token in text.split():
        frequency[token] += 1
```




In [87]:
# A: remember that documents is a list
frequency = defaultdict(int)
for text in documents:
    for token in text.split():
        frequency[token] += 1
frequency

defaultdict(int,
            {'Brocolli': 1,
             'is': 2,
             'good': 3,
             'to': 6,
             'eat.': 1,
             'My': 2,
             'brother': 3,
             'likes': 1,
             'eat': 1,
             'brocolli,': 1,
             'but': 2,
             'not': 1,
             'my': 4,
             'mother.': 1,
             'mother': 2,
             'spends': 1,
             'a': 1,
             'lot': 1,
             'of': 1,
             'time': 1,
             'driving': 2,
             'around': 1,
             'baseball': 1,
             'practice.': 1,
             'Some': 1,
             'health': 1,
             'experts': 1,
             'suggest': 1,
             'that': 2,
             'may': 1,
             'cause': 1,
             'increased': 1,
             'tension': 1,
             'and': 1,
             'blood': 1,
             'pressure.': 1,
             'I': 1,
             'often': 1,
             'feel': 1,
           

**7.2 Remove any words that only appear once, or appear in the stopwords.**

Iterate through the documents and only keep useful words/tokens.

In [102]:
# A:
# list of delete keys 
key_del = [] 
for key, val in frequency.items(): 
    if val == 1 or key in stopwords.words('english'): 
        key_del.append(key) 
          
for i in key_del: 
    del frequency[i] 
      
# Modified Dictionary      
print(frequency) 

defaultdict(<class 'int'>, {'good': 3, 'My': 2, 'brother': 3, 'mother': 2, 'driving': 2})


**7.3 Create the `corpora.Dictionary` object with the retained tokens.**

In [104]:
# A: input has to be a list format
dict = corpora.Dictionary([frequency.keys()])
dict.token2id

{'My': 0, 'brother': 1, 'driving': 2, 'good': 3, 'mother': 4}

**7.4 Use the `dictionary.doc2bow()` function to convert the texts to bag-of-word representations.**

In [109]:
# A: corpus is the BOW representation
corpus = [dictionary.doc2bow(doc) for doc in [frequency.keys()]]
corpus

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)]]

**Why should we use this process?**

The main advantage is that this dictionary object has quick helper functions.

However, there are also some major performance advantages if you ever want to save your model to a file, then load it at a later time.  Tokenizations can take a while to be computed, especially when your text files are quite large. You can save these post-computed dictionary items to file, then load them from disk later which is quite a bit faster.  Also, it's possible to add new documents to your corpus without having to re-tokenize your entire set.  This is great for online systems that can take new documents on demmand.  

As you work with larger datasets with text, this is a much better way to handle LDA and other Gensim models from a performance point of view.

### 8. Set up the LDA model

We can create the gensim LDA model object like so:

```python
lda = models.LdaModel(
    # supply our sparse predictor matrix wrapped in a matutils.Sparse2Corpus object
    matutils.Sparse2Corpus(X, documents_columns=False),
    # or alternatively use the corpus object created with the dictionary in the previous frame!
    # corpus,
    # The number of topics we want:
    num_topics  =  3,
    # how many passes over the vocabulary:
    passes      =  20,
    # The id2word vocabulary we made ourselves
    id2word     =  vocab
    # or use the gensim dictionary object!
    # id2word     =  dictionary
)
```

In [126]:
# A:
lda = models.LdaModel(
    # supply our sparse predictor matrix wrapped in a matutils.Sparse2Corpus object
    # matutils.Sparse2Corpus(X, documents_columns=False),
    # or alternatively use the corpus object created with the dictionary in the previous frame!
    corpus,
    # The number of topics we want:
    num_topics  =  3,
    # how many passes over the vocabulary:
    passes      =  20,
    # The id2word vocabulary we made ourselves
    id2word     =  vocab1,
    # or use the gensim dictionary object!
    # id2word     =  dictionary
    random_state = 123
)

### 9. Look at the topics

The model has a `.print_topics` function that accepts the number of topics to print and number of words per topic. The number before the word is the probability of occurance for that word in the topic.

In [123]:
# A:
print(lda.print_topics())
print(lda.show_topics())

[(0, '0.030*"baseball" + 0.030*"blood" + 0.030*"better" + 0.030*"brocolli" + 0.030*"around" + 0.030*"professionals" + 0.030*"spends" + 0.030*"perform" + 0.030*"practice" + 0.030*"pressure"'), (1, '0.030*"brocolli" + 0.030*"better" + 0.030*"around" + 0.030*"blood" + 0.030*"baseball" + 0.030*"professionals" + 0.030*"spends" + 0.030*"perform" + 0.030*"practice" + 0.030*"pressure"'), (2, '0.083*"baseball" + 0.083*"around" + 0.083*"better" + 0.083*"blood" + 0.083*"brocolli" + 0.021*"professionals" + 0.021*"spends" + 0.021*"perform" + 0.021*"practice" + 0.021*"pressure"')]
[(0, '0.030*"baseball" + 0.030*"blood" + 0.030*"better" + 0.030*"brocolli" + 0.030*"around" + 0.030*"professionals" + 0.030*"spends" + 0.030*"perform" + 0.030*"practice" + 0.030*"pressure"'), (1, '0.030*"brocolli" + 0.030*"better" + 0.030*"around" + 0.030*"blood" + 0.030*"baseball" + 0.030*"professionals" + 0.030*"spends" + 0.030*"perform" + 0.030*"practice" + 0.030*"pressure"'), (2, '0.083*"baseball" + 0.083*"around" + 0.

### 10. Get the topic scores for a document

The `.get_document_topics` function accepts a bag-of-words representation for a document and returns the scores for each topic.

In [127]:
# A:
print(lda.get_document_topics(corpus[0]))
print(lda.get_document_topics(corpus[0]))

[(0, 0.05613149), (1, 0.88773704), (2, 0.056131504)]
[(0, 0.056131545), (1, 0.887737), (2, 0.056131504)]


### 11. Label and visualize the topics

Lets come up with some high level labels. This is the subjective part of LDA. What do the word probabilties that represent topics mean?  Let's make some up.

Plot a heatmap of the topic probabilities for each of the documents.

In [17]:
# A:


### 12. Fit an LDA model with sklearn

Sklearn's LDA model is in the decomposition submodule:

```python
from sklearn.decomposition import LatentDirichletAllocation
```

One of the greatest benefits of the sklearn implementation is that it comes with the familiar `.fit()`, `.transform()` and `.fit_transform()` methods.

**12.1 Initialize and fit an sklearn LDA with `n_topics=3` on our output from the CountVectorizer.**

In [18]:
# A:

**12.2 Print out the topic-word distributions using the `.components_` attribute.**

Each row of this matrix represents a topic, and the columns are the words. (These are not probabilities).

In [19]:
# A:

**12.3 Use the `.transform()` method to convert the matrix into the topic scores.**

These are the document-topic distributions.

In [20]:
# A:

### 13. Further steps

This has been a very basic example.  LDA typically doesn't perform well on very small datasets.  You should try to see how it behaves on your own using a larger text dataset.  Keep in mind: finding the optimal number of topics can be tricky and subjective.

**Generally, you should consider:**
- How well topics are applied to documents overall
- The strength of topics overall, to all documents
- Improving preprocessing such as stopword removal
- Building a nice web interface to explore your documents (see: [LDAExplorer](https://github.com/dyerrington/LDAExplorer), and [pyLDAvis](https://github.com/bmabey/pyLDAvis/blob/master/README.rst))

These general guidelines should help you tune your hyperparameter **K** for number of topics.